### Imports

In [ ]:
import os
import time

import numpy as np
import pandas as pd
import requests_cache
from openmeteo_requests import Client
from resolve_path import ajuste_path, read_input

### Constantes

### Leitura dos Dados

In [ ]:
pathUtil = ajuste_path('data/util/')
pathInput = ajuste_path('data/input/')

df_treinamento = pd.read_csv(
    pathUtil + 'dataset_treinamento.csv').reset_index()

### Definir data de início e fim da API

In [ ]:
ano_mes_ultimo = df_treinamento['ano_mes'].min() + '-01'
ano_mes_primeiro = df_treinamento['ano_mes'].max() + '-30'

START_DATE = ano_mes_ultimo
END_DATE = ano_mes_primeiro

print(ano_mes_primeiro)
print(START_DATE)

### Agrupar o dataset de treinamento por Latitude e Longitude

In [ ]:
df_treinamento['latitude_api'] = df_treinamento['latitude']
df_treinamento['longitude_api'] = df_treinamento['longitude']

for index, row in df_treinamento[df_treinamento['latitude_acidente'].notna()].iterrows():
    df_treinamento.loc[index, 'latitude_api'] = row['latitude_acidente']
    df_treinamento.loc[index, 'longitude_api'] = row['longitude_acidente']

### Retirar linhas onde latitude ou longitude é nan

In [ ]:
df_treinamento = df_treinamento[(df_treinamento['latitude_api'].notna()) & (
    df_treinamento['longitude_api'].notna())]

### Arredonda coordenadas para melhor entrada da API

In [ ]:
df_treinamento['latitude_api'] = df_treinamento['latitude_api'].apply(
    lambda x: round(x, 7))
df_treinamento['longitude_api'] = df_treinamento['longitude_api'].apply(
    lambda x: round(x, 7))

### DF de valores únicos de pares de latitude e longitude

In [ ]:
df_treinamento_agrupado = df_treinamento.groupby(
    ['latitude_api', 'longitude_api'])

### Verifica para quais coordenadas devem ser feitas as requisições

In [ ]:
# Verifica se o csv dados_meteorologicos.csv já existe
if os.path.exists(pathInput + 'dataset_dados_meteorologicos.csv'):
    df_resultados = pd.read_csv(pathInput + 'dataset_dados_meteorologicos.csv')
    # Verifica quais coordenadas estão faltando ao comparar com df_treinamento_agrupado
    df_resultados_agrupado = df_resultados.groupby(
        ['latitude_api', 'longitude_api'])
    df_treinamento_agrupado = df_treinamento_agrupado.filter(
        lambda x: (x['latitude_api'].iloc[0], x['longitude_api'].iloc[0]
                   ) not in df_resultados_agrupado.groups
        # Seleciona os valores unicos de latitude e longitude que sobraram
    ).drop_duplicates(subset=['latitude_api', 'longitude_api']).groupby(['latitude_api', 'longitude_api'])
else:
    df_resultados = pd.DataFrame()

### Requisição da API 

#### Criação de funções:

In [ ]:
def obtem_dados_api(latitude: float, longitude: float, start_date=START_DATE, end_date=END_DATE, max_retry=3):
    '''
    Obtém os dados meteorológicos diários de uma determinada coordenada geográfica (latitude, longitude) da API Open-Meteo ("https://archive-api.open-meteo.com/v1/archive").

    Args:
        latitude (float): Latitude da localização.
        longitude (float): Longitude da localização.
        start_date (str): Data de início no formato 'YYYY-MM-DD'.
        end_date (str): Data de término no formato 'YYYY-MM-DD'.
        max_retry (int): Número máximo de tentativas em caso de falha na requisição.

    Returns:
        pd.DataFrame: DataFrame contendo os dados meteorológicos diários.
        None: Retorna None se todas as tentativas de requisição falharem.
    '''
    # Configurar a sessão da API com cache
    cache_session = requests_cache.CachedSession('.cache', expire_after=-1)
    openmeteo = Client(session=cache_session)
    url_api = "https://archive-api.open-meteo.com/v1/archive"

    params = {
        "latitude": latitude,
        "longitude": longitude,
        "start_date": start_date,
        "end_date": end_date,
        "daily": ["weather_code", "temperature_2m_max", "temperature_2m_min", "apparent_temperature_max", "apparent_temperature_min", "precipitation_sum", "precipitation_hours", "wind_speed_10m_max", "wind_gusts_10m_max", "et0_fao_evapotranspiration"],
        "timezone": "America/Sao_Paulo"
    }

    count_retry = 0

    while count_retry < max_retry:
        # Fazer a solicitação da API
        try:
            response = openmeteo.weather_api(url_api, params=params)[0]
            print(
                f"Coordenadas {response.Latitude()}°N {response.Longitude()}°E")
            print(
                f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")

            # Processamento dos dados diários
            daily = response.Daily()

            daily_weather_code = daily.Variables(0).ValuesAsNumpy()
            daily_temperature_2m_max = daily.Variables(1).ValuesAsNumpy()
            daily_temperature_2m_min = daily.Variables(2).ValuesAsNumpy()
            daily_apparent_temperature_max = daily.Variables(3).ValuesAsNumpy()
            daily_apparent_temperature_min = daily.Variables(4).ValuesAsNumpy()
            daily_precipitation_sum = daily.Variables(5).ValuesAsNumpy()
            daily_precipitation_hours = daily.Variables(6).ValuesAsNumpy()
            daily_wind_speed_10m_max = daily.Variables(7).ValuesAsNumpy()
            daily_wind_gusts_10m_max = daily.Variables(8).ValuesAsNumpy()
            daily_et0_fao_evapotranspiration = daily.Variables(
                9).ValuesAsNumpy()

            daily_data = {"date": pd.date_range(
                start=pd.to_datetime(daily.Time(), unit="s", utc=True),
                end=pd.to_datetime(daily.TimeEnd(), unit="s", utc=True),
                freq=pd.Timedelta(seconds=daily.Interval()),
                inclusive="left"
            )}

            daily_data["latitude_api"] = latitude
            daily_data["longitude_api"] = longitude
            daily_data["weather_code"] = daily_weather_code
            daily_data["temperature_2m_max"] = daily_temperature_2m_max
            daily_data["temperature_2m_min"] = daily_temperature_2m_min
            daily_data["apparent_temperature_max"] = daily_apparent_temperature_max
            daily_data["apparent_temperature_min"] = daily_apparent_temperature_min
            daily_data["precipitation_sum"] = daily_precipitation_sum
            daily_data["precipitation_hours"] = daily_precipitation_hours
            daily_data["wind_speed_10m_max"] = daily_wind_speed_10m_max
            daily_data["wind_gusts_10m_max"] = daily_wind_gusts_10m_max
            daily_data["et0_fao_evapotranspiration"] = daily_et0_fao_evapotranspiration

            return pd.DataFrame(data=daily_data)

        except Exception as e:
            print(
                f"Erro ao solicitar dados da API para ({latitude}, {longitude}): {e}")
            if 'Minutely API request limit exceeded' in str(e):
                print(
                    "Limite de requisições por minuto excedido. Tentando novamente em um minuto...")
                # Espera por 60 segundos antes de tentar novamente
                time.sleep(60)
                count_retry += 1
            elif 'Hourly API request limit exceeded' in str(e):
                print(
                    "Limite de requisições por hora excedido. Tentando novamente em uma hora...")
                # Espera por 3600 segundos (1 hora) antes de tentar novamente
                time.sleep(3600)
                count_retry += 1
            else:
                break

    return None


def agrupa_e_concatena_dados(df_iteracao: pd.DataFrame, df_resultados: pd.DataFrame):
    """
    Agrupa os dados de iteração por ano e mês, calcula a média de cada coluna e concatena com o DataFrame de resultados.

    Args:
        df_iteracao (pd.DataFrame): DataFrame contendo os dados meteorológicos de uma iteração.
        df_resultados (pd.DataFrame): DataFrame contendo os dados meteorológicos acumulados.

    Returns:
        pd.DataFrame: DataFrame resultante da concatenação dos dados agrupados de iteração com os dados acumulados.
    """
    # Extrair Mês e Ano do DatetimeIndex
    df_iteracao['ano'] = df_iteracao['date'].dt.year
    df_iteracao['mes'] = df_iteracao['date'].dt.month

    # Agrupar os dados por 'ano' e 'mes', e calcula a média de cada coluna
    df_iteracao = df_iteracao.groupby(['ano', 'mes']).mean().reset_index()

    return pd.concat([df_resultados, df_iteracao], ignore_index=True)

#### Chamada da API e geração do dataset meteorológico

In [ ]:
# Verifica se há requisições a fazer
total_de_requisicoes = len(df_treinamento_agrupado)
print("Número de requisições a fazer:", total_de_requisicoes)
count = 1

if len(df_treinamento_agrupado) > 0:
    # iterar sobre cada grupo de coordenadas
    for (latitude_api, longitude_api), group in df_treinamento_agrupado:
        print("\nRequisição", count, "de", total_de_requisicoes)
        local = group['local_de_instalacao'].iloc[0]
        print("Fazendo requisição para:", local)

        # Obter os dados meteorológicos da API
        resposta = obtem_dados_api(latitude_api, longitude_api)
        if isinstance(resposta, pd.DataFrame):
            df_iteracao = resposta

            # Concatenar os dados da iteração com os dados já existentes
            df_resultados = agrupa_e_concatena_dados(
                df_iteracao, df_resultados)

            # Salva o dataset de dados meteorológicos
            df_resultados.to_csv(
                pathInput + 'dataset_dados_meteorologicos.csv', index=False)

        else:
            # Encerrar o loop caso a 'obtem_dados_api' retorne None
            print("Número de retries excedido para", local,
                  "(", latitude_api, ",", longitude_api, ").  Encerrando loop...")
            break

        count += 1
else:
    print("Nenhuma requisição a fazer")

### Join com o dataset de treinamento

In [ ]:
df_treinamento['ano'] = df_treinamento['ano_mes'].str.split('-').str[0]
df_treinamento['mes'] = df_treinamento['ano_mes'].str.split('-').str[1]
df_treinamento.drop(columns='ano_mes', inplace=True)

In [ ]:
df_treinamento['ano'] = df_treinamento['ano'].astype(int)
df_treinamento['mes'] = df_treinamento['mes'].astype(int)

### Dropando coluna "date" da API

In [ ]:
df_resultados.drop(columns=['date'], inplace=True)

### Arredondando valores de latitude e longitude

In [ ]:
df_treinamento['latitude_api'] = df_treinamento['latitude_api'].apply(
    lambda x: round(x, 7))
df_treinamento['longitude_api'] = df_treinamento['longitude_api'].apply(
    lambda x: round(x, 7))


df_resultados['latitude_api'] = df_resultados['latitude_api'].apply(
    lambda x: round(x, 7))
df_resultados['longitude_api'] = df_resultados['longitude_api'].apply(
    lambda x: round(x, 7))

### Merge do dataset de treinamento e resultados da API

In [ ]:
df_treinamento = pd.merge(df_treinamento,
                          df_resultados, on=['latitude_api', 'longitude_api', 'ano', 'mes'], how='left')

### Dropando coluna utilizada para merge 

In [ ]:
df_treinamento.drop(columns=['latitude_api', 'longitude_api'], inplace=True)

### Exporta dataset de treinamento com dados meteorológicos

In [ ]:
# Salva o dataset de treinamento com os dados meteorológicos
df_treinamento.to_csv(pathUtil + 'dataset_treinamento_final.csv', index=False)